# Tutorial de Chamadas Paralelas de Funções com Phi-4 Mini ONNX

Este notebook demonstra como usar o Phi-4 Mini com ONNX Runtime GenAI para chamadas paralelas de funções. As chamadas de funções permitem que o modelo invoque de forma inteligente ferramentas e APIs externas com base nos pedidos do utilizador.

## Visão Geral

Neste tutorial, vais aprender a:
- Configurar o Phi-4 Mini com ONNX Runtime GenAI
- Definir esquemas de funções para reserva de voos e hotéis
- Utilizar geração guiada com gramática Lark para obter resultados estruturados
- Executar chamadas paralelas de funções para cenários complexos de reserva de viagens

## Pré-requisitos

Antes de executar este notebook, certifica-te de que:
- Fizeste o download do modelo Phi-4 Mini ONNX
- Instalaste o pacote `onnxruntime-genai`
- Tens uma compreensão básica dos conceitos de chamadas de funções


## Passo 1: Importar Bibliotecas Necessárias

Primeiro, vamos importar as bibliotecas necessárias para a implementação da chamada de função.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Passo 2: Configuração e Configuração do Modelo

Agora vamos configurar o modelo Phi-4 Mini ONNX. Certifique-se de substituir o caminho pelo diretório real do seu modelo.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Passo 3: Configurar os Parâmetros de Geração

Defina os parâmetros de geração para controlar o comportamento de saída do modelo. Estas configurações garantem respostas determinísticas e focadas para a chamada de funções.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Passo 4: Definir Funções Disponíveis

Aqui definimos as funções que o nosso assistente de IA pode utilizar. Neste exemplo, temos duas funções relacionadas a viagens:
1. **booking_flight_tickets**: Para reservar voos entre aeroportos
2. **booking_hotels**: Para reservar acomodações em hotéis

As definições das funções seguem o formato de esquema de chamadas de função da OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Passo 5: Funções Auxiliares para Geração de Gramática

Estas funções auxiliares convertem as definições das nossas funções para o formato de gramática Lark, utilizado pelo ONNX Runtime GenAI para geração orientada. Isto garante que o modelo produza chamadas de função válidas no formato JSON correto.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Passo 6: Testar a Geração de Gramática

Vamos testar as nossas funções de geração de gramática para verificar como elas convertem as definições das ferramentas no formato adequado.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Passo 7: Preparar o Prompt do Sistema e o Gerador

Agora vamos criar o prompt do sistema que informa ao modelo sobre as ferramentas disponíveis e configurar o gerador com parâmetros de geração orientada.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Passo 8: Inicializar o Gerador com Geração Guiada

Agora vamos criar o gerador com os parâmetros configurados e aplicar a gramática Lark para geração guiada.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Passo 9: Testar Chamadas de Funções em Paralelo

Agora vamos testar a nossa configuração com um cenário complexo de reserva de viagem que exige a chamada de várias funções.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Gerar Chamadas de Função

O modelo agora irá gerar chamadas de função estruturadas com base no pedido do utilizador. Graças à geração orientada, o resultado estará em formato JSON válido que pode ser executado diretamente.

**Resultado Esperado**: O modelo deve gerar chamadas de função para:
1. `booking_flight_tickets` com detalhes de Pequim (PEK) para Paris (CDG)
2. `booking_hotels` com detalhes de alojamento em Paris

Execute a célula abaixo para ver a geração em tempo real:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Conclusão

🎉 **Parabéns!** Conseguiu implementar com sucesso chamadas de funções paralelas com o Phi-4 Mini utilizando o ONNX Runtime GenAI.

### O que Aprendeu:

1. **Configuração do Modelo**: Como configurar o Phi-4 Mini com o ONNX Runtime GenAI  
2. **Definição de Funções**: Como definir esquemas de funções para chamadas de funções de IA  
3. **Geração Guiada**: Como usar a gramática Lark para gerar saídas estruturadas  
4. **Chamadas de Funções Paralelas**: Como lidar com pedidos complexos que requerem múltiplas chamadas de funções  

### Principais Benefícios:

- ✅ **Saída Estruturada**: A geração guiada garante chamadas de funções JSON válidas  
- ✅ **Processamento Paralelo**: Lida com múltiplas chamadas de funções num único pedido  
- ✅ **Alta Performance**: O ONNX Runtime oferece inferência otimizada  
- ✅ **Esquema Flexível**: Fácil adicionar ou modificar definições de funções  

### Recursos:

- [Documentação do Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)  
- [Documentação do ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)  
- [Melhores Práticas para Chamadas de Funções](https://platform.openai.com/docs/guides/function-calling)  



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante notar que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
